# Permutation engine examples

This shows how to use the permutation engine to permute gates of a certain type towards the beginning or end of the circuit.
It relies on the implementation of permutation rules between different gates. Currently, only rotational gates + CNOTs + Pauli + S + H + T and TimeEvolution gates are supported. Arbitrary controlled gates are not supported and must be decomposed first. We provide a handy engine list that translates any circuit into the supported gate set and performes the permutation.

In [7]:
import projectq
import cmath
from projectq.backends import CommandPrinter, Simulator
from projectq.ops import *
from projectq.cengines import PermutePi4Front

#### First generate the main engine and the order of all the engines in the compiler stack.

In [8]:
engines = [CommandPrinter(),projectq.cengines.TagRemover(),PermutePi4Front()]
eng = projectq.MainEngine(backend=CommandPrinter(), engine_list=engines)

Now lets generate a very simple circuit

In [9]:
qubit = eng.allocate_qubit()
X | qubit
Y | qubit
X | qubit
T | qubit
T | qubit
eng.flush()

Allocate | Qureg[0]
Deallocate | Qureg[0]
X | Qureg[0]
Y | Qureg[0]
X | Qureg[0]
T | Qureg[0]
T | Qureg[0]
Allocate | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rx(3.14159265359) | Qureg[0]
Ry(3.14159265359) | Qureg[0]
Rx(3.14159265359) | Qureg[0]


Now lets take a look at a bit more complicated circuit that is still natively supported:

In [10]:
engines2 = [CommandPrinter(),projectq.cengines.TagRemover(),PermutePi4Front()]
eng2 = projectq.MainEngine(backend=CommandPrinter(accept_input=False), engine_list=engines2, verbose=True)

qubit1 = eng2.allocate_qubit()
qubit2 = eng2.allocate_qubit()
qubit3 = eng2.allocate_qubit()
qubit4 = eng2.allocate_qubit()

T | qubit1
CNOT | (qubit2, qubit3)
Rx(-cmath.pi/2) | qubit4

CNOT | (qubit2, qubit1)
Rx(cmath.pi/2) | qubit3
Rz(cmath.pi/4) | qubit4


Rz(cmath.pi/4) | qubit1
Rz(cmath.pi/2) | qubit2
Rz(cmath.pi/4) | qubit3
Rz(cmath.pi/2) | qubit4

Rx(-cmath.pi/2) | qubit1
Rx(cmath.pi/2) | qubit2
Rx(cmath.pi/2) | qubit3
Rx(cmath.pi/2) | qubit4

eng2.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
T | Qureg[0]
CX | ( Qureg[1], Qureg[2] )
Rx(10.995574287564) | Qureg[3]
CX | ( Qureg[1], Qureg[0] )
Rx(1.570796326795) | Qureg[2]
Rz(0.785398163397) | Qureg[3]
Rz(0.785398163397) | Qureg[0]
Rz(1.570796326795) | Qureg[1]
Rz(0.785398163397) | Qureg[2]
Rz(1.570796326795) | Qureg[3]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
Rx(1.570796326795) | Qureg[3]
Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
T | Qureg[0]
Ry(0.785398163397) | Qureg[3]
exp(0.3926990816985j * (1.0 Z0 Z1)) | Qureg[0-1]
exp(0.3926990816985j * (1.0 Y0 Z1)) | Qureg[2, 1]
CX | ( Qureg[1], Qureg[2] )
Rx(10.995574287564) | Qureg[3]
CX | ( Qureg[1], Qureg[0] )
Rx(1.570796326795) | Qureg[2]
Rz(1.570796326795) | Qureg[1]
Rz(1.570796326795) | Qureg[3]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
Rx(1.570796326795) | Qureg[3]

This is actually the circuit shown in the paper: https://arxiv.org/abs/1808.02892

# On to more general circuits...
The provided engine_list is a whole stack of compile engines that should be able to translate an arbitrary circuit into the format needed by arxiv:1808.02892

In [13]:
from projectq.setups.surface_codes import lattice_surgery
engines3 = lattice_surgery.get_engine_list()
print(engines3)

[<projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f7ebbe02cf8>, <projectq.cengines._tagremover.TagRemover object at 0x7f7ebbd7f588>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f7ebbd7f5c0>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f7ebbd7f5f8>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f7ebbd7f630>, <projectq.cengines._tagremover.TagRemover object at 0x7f7ebbd7f668>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f7ebbd7f6a0>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f7ebbd7f6d8>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f7ebbd7f710>, <projectq.cengines._tagremover.TagRemover object at 0x7f7ebbd7f748>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f7ebbd7f780>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f7ebbd7f7b8>, <projectq.cengines._permutation._permutation.PermutePi4Front object at 0x7f7ebbd7f7f0>]


In [14]:
eng3 = projectq.MainEngine(backend=CommandPrinter(accept_input=False), engine_list=engines3)


# now add a general circuit
